In [1]:
import subprocess
from itertools import product
from math import prod
from random import randrange
import random
import numpy as np
import pandas as pd
from multiprocess import Pool

# Комп'ютерний практикум №1

**Виконано студентами**

**групи ФІ-32мн**

**Карловський Володимир**

**Кріпака Ілля**

## Мета лабораторної роботи

Практично ознайомитися із сучасним методом криптоаналізу блокових шифрів, набути навички у дослідженні стійкості блокових шифрів до лінійного криптоаналізу.

## Постановка задачі

|                           Постановка задачі                            |    Зроблено    |
|:----------------------------------------------------------------------:|:--------------:|
|                    Реалізувати функції шифру Хейса                     | $ \checkmark $ |
| Пошук високоімовірних п’ятираундових лінійних апроксимацій шифру Хейса | $ \checkmark $ |
|            Реалізувати атаку на перший раундовий ключ Хейса            | $ \checkmark $ |

## Хід роботи / Опис труднощів
Позаяк другий практикум є логічним продовженням першого, тож реалізація була легшою, але із іншого не менш простою чим у першій лабораторній роботі. Проблеми виникли із:
* написанням функції для перебору потенційних кадидатів у ключі. Як виявилося нічого складного, просто саме у той час наклалися часові проблеми;
* обчислювальними можливостями ноутбука. На пристрої де була запущена атака, використовувалися майже усі наявні можливості із оперативно пам'яттю. Майже завжди під час перебору оперативна пам'ять була заповнена на 50 гігабайт із 64. Звичайно можна було оптимізувати перебір та не використовувати звичайні _list()_, а навпаки _nd_array_, але то може у майбутньому зробимо таке покращення.

## Варіант 5

S = (F, 8, E, 9, 7, 2, 0, D, C, 6, 1, 5, B, 4, 3, A)

## Пояснення алгоритму

![Attack rounds](./data/attack_rounds.png)

![m2_algorithm](./data/m2_algorithm.png)

Сам алгоритм можна розбити на такі частини.

1) Передобчислення таблиці коефіцієнтів кореляції лінійної апроксимації булевої функції для кожного значення $\alpha,
   \beta$.
2) Знаходження високоймовірних п'ятираундових апроксимацій шифру Хейса із великим потенціалом.
3) Знаходження першого раундового ключа:
    1) Створення даних для аналізу потенційних ключів. Тобто на цьому кроці генерується випадковий текст, який буде
       зашифровано двома різними шляхами. Перший із них буде зашифровано лише на один раунд, а другий зашифровано за
       допомогою виконуваного файлу "heys.bin".
    2) Перебір усіх можливих ключів. На цьому кроці використовується атака М2, що проілюстрована на рисунку 2. Загалом
       атаку М2 можна описати наступним чином:
        1) Згенерувати один великий текст для перевірки ключів та зашифрувати його двома способами (один - зашифрувати
           на 1 раунд, другий - зашифрувати шифром Хейса).
       2) Обчислюємо значення лінійної апроксимації і відповідне значення $\overset{u_k}{^}$. 
            Саме цю формулу можна трохи оптимізувати. 
            $$ \overset{u_k}{^} = |(X_1, Y): \{ \alpha \cdot X_1 \oplus \beta \cdot Y = 0 \}| - |( X_1, Y): \{ \alpha \cdot X_1 \oplus \beta \cdot Y = 1 \}| = \text{# cимволів у} (\alpha \cdot X_1 \oplus \beta \cdot Y) - 2 \cdot |(X_1, Y): \{ \alpha \cdot X_1 \oplus \beta \cdot Y = 1 \}| $$
        3) Збираємо певну кількість ключів, що мають максимальне значення $\overset{u_k}{^}$.
        4) У результаті залишиться деяка множина гіпотетичних ключів, перебираючи які можна буде дізнатися правильний
           ключ.

## Порогові значення

* Порогове значення для методу гілок та границь: 0.00015. Чому саме таке значення? Експериментальним шляхом визначили,
  що саме після 0.00015 починає генеруватися дуже багато значень із маленькими ймовірностями.
* Кількість текстів потрібна для атаки: 1 шт у якому було 7000 екземплярів.
* Кількість ключів, що обираються для подальшого аналізу: 200. Це значення було обрано із розрахунку наявних
  обчислювальних можливостей, взагалі можна було і 50 чи 100 взяти.

In [2]:
s_block = [0xF, 0x8, 0xE, 0x9, 0x7, 0x2, 0x0, 0xD, 0xC, 0x6, 0x1, 0x5, 0xB, 0x4, 0x3, 0xA]
s_block_inverse = [0x6, 0xA, 0x5, 0xE, 0xD, 0xB, 0x9, 0x4, 0x1, 0x3, 0xF, 0xC, 0x8, 0x7, 0x2, 0x0]
ENC = True
DEC = False
MAX_16BIT_NUM = (1 << 16) - 1
VARIANT = 5


class heys:

    def heys_round(self, n, key=0):
        ct = n ^ key
        ct = self.substitute(ct, s_block)
        ct = self.mix_words(ct)
        return ct

    # r = (s_1(y_1), s_2(y_2), ... , s_n(y_n)) 
    def substitute(self, n, enc):
        s = s_block if enc == True else s_block_inverse
        r = 0
        for i in range(4):
            r |= s[(n >> (i * 4)) & 15] << (i * 4)
        return r

    # i-тий біт j-того фрагменту стає j-тим бітом i-того фрагменту.
    def mix_words(self, n):
        r = 0
        for j in range(4):
            for i in range(4):
                r |= (n >> (4 * j + i) & 1) << (4 * i + j)
        return r


## Створення таблиці лінійних потенціалів

In [3]:
heys_obj = heys()
# таблиця лінійних потенціалів для перестановки шифру Хейса
linear_potential = np.zeros((16, 16))


# makes AND operation to get what bits has to be XORed and performs XOR of all 
#  variables by taking weight of number and mod 2
def custom_mul(x: int, y: int):
    return (x & y).bit_count() % 2


for alpha in range(16):
    for beta in range(16):
        for x in range(16):
            linear_potential[alpha][beta] += (-1) ** (
                    custom_mul(alpha, x) ^ custom_mul(beta, heys_obj.substitute(x, ENC)))
linear_potential /= 16
linear_potential **= 2

pd.DataFrame(linear_potential)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000
2,0.0,0.0625,0.0625,0.2500,0.0625,0.0000,0.0000,0.0625,0.0000,0.0625,0.0625,0.2500,0.0625,0.0000,0.0000,0.0625
3,0.0,0.2500,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625
4,0.0,0.0000,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.2500,0.0000,0.0000,0.0625,0.0625
5,0.0,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.2500,0.0625,0.0000,0.0625
6,0.0,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.0625,0.2500,0.0625,0.2500
7,0.0,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.2500,0.2500
8,0.0,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625
9,0.0,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000,0.0625,0.0000,0.2500,0.0625,0.0625,0.0000,0.0000,0.0625


In [4]:
PROBABILITY_THRESHOLD = 0.00015
local_diff_table = {}


# betas
def get_new_betas(alpha):
    # obtain raw alpha values from one big Alpha
    alpha_list = [(alpha >> (4 * i)) & 15 for i in range(4)]
    # obtain non-null betas from s block diff table
    non_null_betas = []
    for i, alpha in enumerate(alpha_list):
        # filter betas with non-zero entries
        betas = [beta for beta in range(16) if linear_potential[alpha][beta] != 0]
        non_null_betas.append(betas)

    betas = {}
    # generate all possible combinations that can appear in resulted beta
    for (i, beta_list) in enumerate(product(*non_null_betas, repeat=1)):
        beta = sum([beta_list[i] << (i * 4) for i in range(4)])
        betas[beta] = prod([linear_potential[alpha_list[i]][beta_list[i]] for i in range(4)])

    return betas


def differential_search(alpha):
    # shared between workers variable
    global local_diff_table

    # Г_{t-1}(alpha) (Г spells like 'g')
    g_prev = {alpha: 1}
    # like in rust -- 1..=5
    for i in range(1, 6):
        g_current = {}
        # misuse of 'beta' naming (by now we are iterating over last alphas right before calculation of probabilities)
        for beta, p_i in g_prev.items():

            if beta not in local_diff_table:
                local_diff_table[beta] = get_new_betas(beta)

            # extract possible candidates for specific beta ('alpha') and iterate over them
            for gamma in local_diff_table[beta]:
                tmp_prob = p_i * local_diff_table[beta][gamma]
                if gamma not in g_current:
                    g_current[gamma] = tmp_prob
                else:
                    g_current[gamma] += tmp_prob

        # Г_{t}(alpha)
        g_new_filtered = {}
        #check "bounds" and write to new list mixed words (aka gammas are becoming new alphas)
        for gamma in g_current.keys():
            if g_current[gamma] > PROBABILITY_THRESHOLD:
                g_new_filtered[heys_obj.mix_words(gamma)] = g_current[gamma]

        g_prev = g_new_filtered

    return (alpha, g_prev)

## Знайдені за допомогою методу «гілок та границь» високоймовірних ланійних апроксимацій
Вони наведені у формі $(\alpha \rightarrow [(\beta, prob)  \dots ], \dots )$

In [5]:
%%time

def prepare_alphas():
    alfas = []
    for i in range(4):
        for j in range(1, 16):
            alfas.append(j << (4 * i))
    return alfas


def init_worker(shared_diff_table):
    global local_diff_table
    local_diff_table = shared_diff_table


with Pool(initializer=init_worker, initargs=(local_diff_table,)) as pool:
    candidates = list(
        pool.map(
            differential_search,
            prepare_alphas()
        )
    )

print('Possible candidates after "branches and bounds" search, candidates len: {0}\n candidates: \n'.format(
    len(candidates)))
list(candidates)

Possible candidates after "branches and bounds" search, candidates len: 60
 candidates: 

CPU times: user 353 ms, sys: 57.2 ms, total: 411 ms
Wall time: 1min 32s


[(1,
  {1: 0.00010913595906458795,
   17: 0.00019877473823726177,
   257: 0.0002441936812829226,
   272: 0.0002748606784734875,
   4097: 9.778328239917755e-05,
   4112: 0.00010361778549849987,
   4352: 0.00023001930094324052,
   4353: 0.00010770518565550447,
   4368: 0.0001497188932262361,
   4113: 0.0001487565750721842,
   4096: 0.00017048648442141712,
   1028: 0.00010890141129493713,
   1088: 0.00010890141129493713,
   8: 0.00014866283163428307,
   136: 0.0002573244273662567,
   2056: 0.0003977044252678752,
   2176: 0.000412687542848289,
   32776: 0.00016441941261291504,
   32896: 0.00014666840434074402,
   34816: 0.0003159641055390239,
   34824: 0.0001610609469935298,
   34944: 0.00021610211115330458,
   32904: 0.00020184554159641266,
   34952: 0.00012446928303688765,
   32768: 0.00024058064445853233}),
 (2,
  {1: 0.0001206029555760324,
   17: 0.00023010544828139246,
   257: 0.00020125412265770137,
   272: 0.0002335640019737184,
   4112: 0.00010085743269883096,
   4352: 0.0002478646

## Відфільтровані кандидати 
Вони наведені у формі $((\alpha, \beta), prob)$

In [7]:
filtered_candidates = []
for alpha, betas in list(candidates):
    for beta, prob in betas.items():
        filtered_candidates.append(((alpha, beta), prob))
filtered_candidates = sorted(filtered_candidates, key=lambda x: x[1], reverse=True)
filtered_candidates[:30]

[((1, 2176), 0.000412687542848289),
 ((3, 32768), 0.0004049427807331085),
 ((1, 2056), 0.0003977044252678752),
 ((11, 32768), 0.0003616251051425934),
 ((16, 257), 0.00033808920488809235),
 ((32, 4352), 0.00033525656908750534),
 ((16, 272), 0.0003321608528494835),
 ((2, 34816), 0.0003316223155707121),
 ((3, 34816), 0.0003307648003101349),
 ((2, 2176), 0.00032900902442634106),
 ((9, 34824), 0.0003206683322787285),
 ((3, 34824), 0.00032046809792518616),
 ((12, 34816), 0.0003159772604703903),
 ((1, 34816), 0.0003159641055390239),
 ((48, 4096), 0.00031182216480374336),
 ((15, 2176), 0.0003100536996498704),
 ((14, 32768), 0.00030848116148263216),
 ((13, 32768), 0.0003076973371207714),
 ((32, 272), 0.0003004154823429417),
 ((13, 8), 0.0002996046096086502),
 ((14, 8), 0.00029633427038788795),
 ((2, 2056), 0.00029422040097415447),
 ((12, 34824), 0.00029390817508101463),
 ((9, 34952), 0.0002918709069490433),
 ((176, 4096), 0.0002914851647801697),
 ((224, 1), 0.0002904195134760812),
 ((16, 4352),

## Перевірка ключів

In [7]:
%%time

# 1/0.00015 = 6666 ~ 7000
# 7000 = 8 * 875
SAMPLES_NUM = 7000
# checking keys from 0..=((1<<16)-1)
keys = [i for i in range(0, MAX_16BIT_NUM + 1)]


def get_unique_alpha_beta_values(candidates):
    return set([alpha for (alpha, _), _ in candidates]).union(set([beta for (_, beta), _ in candidates]))


# precompute values for computing u(k) value
precomputed_custom_mul = dict(
    [(x, [custom_mul(x, k) for k in keys]) for x in get_unique_alpha_beta_values(filtered_candidates)])


# text -- 16bit numbers
def write_file(file, text: list[int]):
    with open(file, 'wb') as f:
        for x in text:
            f.write(x.to_bytes(2, byteorder='little'))
        f.close()


def read_file(file):
    text = []
    with open(file, 'rb') as f:
        data = f.read()
        for pair in zip(data[::2], data[1::2]):
            el1, el2 = pair
            text.append((el2 << 8) | el1)
        f.close()
    return text


def get_plaintext_filename(variant: int, index: int):
    return './key_check/pt_{0}_{1}.bin'.format(variant, index)


def get_ciphertext_filename(variant: int, index: int):
    return './key_check/ct_{0}_{1}.bin'.format(variant, index)


def generate_ciphertext(samples_num, index):
    random.seed(0)
    plaintext = [randrange(0, MAX_16BIT_NUM + 1) for _ in range(0, samples_num)]

    filename_pt = get_plaintext_filename(VARIANT, index)
    filename_ct = get_ciphertext_filename(VARIANT, index)
    write_file(filename_pt, plaintext)

    subprocess.Popen('../lab1/data/heys.bin e {0} {1} {2}'.format(VARIANT, filename_pt, filename_ct),
                     shell=True,
                     stdout=subprocess.DEVNULL, stderr=None).communicate(timeout=10)
    ciphertext = read_file(filename_ct)

    with Pool() as pool:
        heys_1_round_enc = list(
            pool.map(lambda k: [heys_obj.mix_words(heys_obj.substitute(x ^ k, ENC)) for x in plaintext], keys))

    return heys_1_round_enc, ciphertext


# \overset{u_k}{^} = |(X_1, Y): \{ \alpha \cdot X_1 \oplus \beta \cdot Y = 0 \}| - |( X_1, Y): \{ \alpha \cdot X_1 \oplus \beta \cdot Y = 1 \}| = \text{# символів у} (\alpha \cdot X_1 \oplus \beta \cdot Y) - 2 \cdot |(X_1, Y): \{ \alpha \cdot X_1 \oplus \beta \cdot Y = 1 \}|
def check_candidate(heys_1_round_enc_texts, ct, alpha, beta):
    u_k = 0
    for (ct1, ct2) in zip(heys_1_round_enc_texts, ct):
        u_k += precomputed_custom_mul[alpha][ct1] ^ precomputed_custom_mul[beta][ct2]
    return abs(SAMPLES_NUM - 2 * u_k)

CPU times: user 223 ms, sys: 9.88 ms, total: 233 ms
Wall time: 232 ms


In [8]:
%%time 
CANDIDATES_THRESHOLD = 200

heys_1_round_enc, ciphertext = generate_ciphertext(SAMPLES_NUM, 1)
possible_key_candidates = {}
for (alpha, beta), _ in filtered_candidates:
    with Pool() as pool:
        success_rates = list(pool.map(lambda k: check_candidate(heys_1_round_enc[k], ciphertext, alpha, beta), keys))

        candidates_taken = CANDIDATES_THRESHOLD
        possible_max_candidates = []
        for max_u_k_value in range(max(success_rates), 0, -1):
            keys_with_max_u_k = [k for k, u_k in enumerate(success_rates) if u_k == max_u_k_value]

            if len(keys_with_max_u_k) <= candidates_taken:
                possible_max_candidates += keys_with_max_u_k
                candidates_taken -= len(keys_with_max_u_k)
            else:
                possible_max_candidates += random.sample(keys_with_max_u_k, candidates_taken)
                break

        possible_key_candidates[(alpha, beta)] = possible_max_candidates

CPU times: user 1min 56s, sys: 13min 48s, total: 15min 45s
Wall time: 38min 45s


## Результат атаки

Отримали наступний список із потенційними ключами серед яких є правильний ключ.

In [15]:
sorted_key_candidates = {}

for (_, _), keys in possible_key_candidates.items():
    for key in keys:
        if key in sorted_key_candidates:
            sorted_key_candidates[key] += 1
        else:
            sorted_key_candidates[key] = 1

sorted_key_candidates = sorted(sorted_key_candidates.items(), key=lambda x: x[1], reverse=True)
[(hex(key), count) for key, count in sorted_key_candidates[:100]]

[('0x2f43', 12),
 ('0x2d43', 11),
 ('0x2d47', 8),
 ('0x5f43', 8),
 ('0xaf43', 8),
 ('0x2543', 8),
 ('0x2943', 8),
 ('0x2e43', 8),
 ('0x5543', 8),
 ('0x2143', 8),
 ('0x6d43', 8),
 ('0xff43', 8),
 ('0xc043', 8),
 ('0x2743', 8),
 ('0xf43', 7),
 ('0x2b43', 7),
 ('0xbf43', 7),
 ('0x6f43', 7),
 ('0x5a43', 7),
 ('0x2c43', 7),
 ('0x4343', 7),
 ('0x6143', 7),
 ('0x7f43', 7),
 ('0xf243', 7),
 ('0xc143', 7),
 ('0x2df3', 7),
 ('0x2d48', 7),
 ('0x2d44', 7),
 ('0x2d4b', 7),
 ('0x2f4f', 7),
 ('0x2d4e', 7),
 ('0x340d', 7),
 ('0x2947', 6),
 ('0x2f47', 6),
 ('0x2d4d', 6),
 ('0x34cc', 6),
 ('0x9f43', 6),
 ('0x1a43', 6),
 ('0xba43', 6),
 ('0x5643', 6),
 ('0x643', 6),
 ('0xef43', 6),
 ('0xce43', 6),
 ('0x2643', 6),
 ('0x743', 6),
 ('0x1243', 6),
 ('0xc243', 6),
 ('0xc543', 6),
 ('0x5b43', 6),
 ('0x5e43', 6),
 ('0xb743', 6),
 ('0xbd43', 6),
 ('0x943', 6),
 ('0xc943', 6),
 ('0x143', 6),
 ('0x6043', 6),
 ('0x2843', 6),
 ('0xf943', 6),
 ('0x7d43', 6),
 ('0x9743', 6),
 ('0x1f43', 6),
 ('0xc343', 6),
 ('0x6743',

## Висновки

За допомогою реалізації другого практикуму із блокових шифрів по знаходженню останньго раундового ключа для шифру Хейса
дізналися на практиці, як проводяться атаки даного штибу для більших криптосистем.